# Task 

Use `GitHub's REST API v3` to find the top 10 contributors across https://github.com/awslabs/ open source repositories, based on the sum of their contributions to each repository.

# Import packages

In [317]:
import pandas as pd
import requests

# Get your token 

- Step 1: Login GitHub with your user name and password
- Step 2: Click your icon on the right top and click "settings"
- Step 3: Scroll down and you can see "Developer settings"
- Step 4: Then click "personal access token " on a new page then click "Generate New Token"
- Step 5: It will require your GitHub password
- Step 6: After you enter your password you can see a new page, and on the top, you can see "Note". Here you can name your token 
- Step 7: click what you want under the "Note"
- Step 8: Scroll down to the bottom and click "Generate Token"

In [306]:
# set up user name we want to research and get your own access_token , then create header 
user='awslabs'
access_token='0250a3f9e9d287fab9f160e3109daadb93e7c072' 
headers = {'Authorization':"Token "+access_token} # we need header to obtain more data 

  # Extract all the repos from host/user - awslabs

In [270]:
# loop through all pages to obtian all the repose
repos=[]
for page_num in range(1,300):
    try:
    # to find all the repos' names from each page
        url=f"https://api.github.com/users/{user}/repos?page={page_num}" 
        repo=requests.get(url,headers=headers).json()
        repos.append(repo)
    except:
        repos.append(None)

In [273]:
# I don't want to run this serval times so i just save repos locally
import pickle
pickle_out=open('repos', 'wb')
pickle.dump(repos, pickle_out)
pickle_out.close()


In [ ]:
pickle_in_data = open("repos","rb")
repos = pickle.load(pickle_in_data)

In [336]:
# Okey now we obtained all the repos under the user name 

for page in repos:
    if page==[]:
        print(repos.index(page))
        break
# there are 17 pages

17


In [339]:
#first page has 30 repose
len(repos[1])

30

In [289]:
# after we obtianed all the repos information under the awslab, next let us get the repos full_name 
all_repo_names=[]
for page in repos:
    for repo in page:
        try:
            all_repo_names.append(repo['full_name'].split("/")[1])
        except:
            pass
  

In [341]:
len(all_repo_names)

494

In [307]:
# collect all contributors from each repo
all_contributors=[]
for repo_name in all_repo_names:
    url = f"https://api.github.com/repos/awslabs/{repo_name}/contributors"

# make the request and return the json
    contributors= requests.get(url,headers=headers).json()
    all_contributors.append(contributors)

In [343]:
len(all_contributors) # total 494 repos and extracted all contributors from each repo 

494

In [310]:
#save it again using pickle 
pickle.dump(all_contributors, open('all_contributors', 'wb'))
pickle_out.close()

In [323]:
all_contributors = pickle.load(open("all_contributors","rb"))

In [311]:
# collect contributors names and their contribution count 
names=[]
counts=[]
for repo in all_contributors:
    for contributor in repo:
        try:
            name=contributor['login']
            count=contributor['contributions']
            names.append(name)
            counts.append(count)
        except:
            None

In [312]:
len(names) # there are total 3501  

3501

In [313]:
# create a dataframe and store the contrbutors' names and their contribution counts
mydata=pd.DataFrame()
mydata['contributor_name']=names
mydata['counts']=counts

In [ ]:
# Then obtain unique names with sum of their contribution counts
mydata=mydata.groupby('contributor_name')["counts"].sum().reset_index().sort_values(by='counts',ascending=False)

In [314]:
# drop None / missing values 
mydata=mydata.dropna(axis=0).reset_index().drop(columns='index')

In [325]:
mydata.head(10) # top 10 contributors 

,contributor_name,counts
0,fchollet,2228
1,timmattison,1645
2,IanMeyers,1231
3,frankfliu,999
4,igorborgest,775
5,dependabot-preview[bot],751
6,jamesiri,698
7,justinboswell,592
8,JonathanHenson,580
9,dwmw2,563


In [326]:
# save mydata
mydata.to_csv("my_data.csv")

#### There might be better ways of ETL. Here are some links of documents/ materials,which might be helpful. 
- Pagination - https://developer.github.com/v3/
- AWS Labs organization - https://github.com/awslabs/
- Personal Access Token - https://docs.github.com/en/github/authenticating-to-github/creating-a-personal-access-token
- Obtian all repos of the owner - https://api.github.com/users/{owner}/repos
- a repository's contributors can be fetched from -  https://api.github.com/repos/{owner}/{repo}/contributors.
- header - headers = {'Authorization':"Token "+access_token} 
- requests with header -  requests.get(url,headers=headers).json()